In [35]:
import numpy as np
from typing import Tuple, Union

# Iterative Techniques in Matrix Algebra

## Chapter 7.3 The Jacobi and Gauss-Siedel Iterative Techniques

### Jacobi Method
$$x^{(k)}_i=\sum^n_{\substack{j\neq i \\ j=1}}\biggl(-\frac{a_{ij}x^{(k)}_{j}}{a_{ii}}\biggr)+\frac{b_i}{a_{ii}}$$

In [36]:
def Jacobi(A:np.ndarray, x:np.ndarray, b:np.ndarray, iter:int, tol:float=1e-5) -> np.ndarray:
    x_ = np.copy(x)
    for iter in range(iter):
        prev_x = np.copy(x_)
        for idx in range(A.shape[0]):
            mask = np.arange(len(x_)) != idx
            x_[idx] = 1 / (A[idx][idx]) * (b[idx] - np.sum(A[idx][mask] @ prev_x[mask]))
        # if np.linalg.norm(x_ - prev_x) < tol:
        #     print("Iteration stopped, norm difference < tolerance.")
        #     break
    return x_

### Gauss-Siedel Method
$$x^{(k)}_i=\frac{1}{a_{ii}}\biggl[
    -\sum^{i-1}_{j=1}(a_{ij}x^{(k)}_j)-\sum^{n}_{j=i+1}(a_{ij}x^{(k-1)}_j)+b_i
\biggr]$$


In [37]:
def GaussSeidel(A:np.ndarray, x:np.ndarray, b:np.ndarray, iter:int, tol:float=1e-5) -> np.ndarray:
    x_ = np.copy(x)
    for iter in range(iter):
        prev_x = np.copy(x_)
        for idx in range(A.shape[0]):
            mask = np.arange(len(x_)) != idx
            x_[idx] = 1 / (A[idx][idx]) * (b[idx] - np.sum(A[idx][mask] @ x_[mask]))
        # if np.linalg.norm(x_ - prev_x) < tol:
        #     print("Iteration stopped, norm difference < tolerance.")
        #     break
    return x_

## Chapter 7.4 Relaxation Techniques for Solving Linear Systems

### Sucessive Over-Relaxation Method
$$x^{(k)}_i=(1-\omega)x^{(k-1)}_i+\frac{\omega}{a_{ii}}\biggl[
    -\sum^{i-1}_{j=1}(a_{ij}x^{(k)}_j)-\sum^{n}_{j=i+1}(a_{ij}x^{(k-1)}_j)+b_i
\biggr]$$
where $\omega > 1$

In [38]:
def SOR(A:np.ndarray, x:np.ndarray, b:np.ndarray, iter:int, omega=1.5, tol:float=1e-5) -> np.ndarray:
    x_ = np.copy(x)
    for iter in range(iter):
        prev_x = np.copy(x_)
        for idx in range(A.shape[0]):
            mask = np.arange(len(x_)) != idx
            x_[idx] = (1 - omega) * prev_x[idx] + (omega / (A[idx][idx]) * (b[idx] - np.sum(A[idx][mask] @ x_[mask])))
        # if np.linalg.norm(x_ - prev_x) < tol:
        #     print("Iteration stopped, norm difference < tolerance.")
        #     break
    return x_

## Chapter 7.6 The Conjugate Gradient Method

In [93]:
def ConjugateGradient(A:np.ndarray, x:np.ndarray, b:np.ndarray, v:np.ndarray, tol=1e-6) -> np.ndarray:
    x_ = x.copy().astype(float)
    for k in range(v.shape[0]):
        t = np.dot(v[:, k], b - A @ x_) / (np.dot(v[:, k], A @ v[:, k]))
        x_ += (t * v[:, k]).reshape(x_.shape)
        if np.linalg.norm(b - A @ x_) < tol:
            return x_
    return x_

def ConjugateGradient_(A:np.ndarray, x:np.ndarray, b:np.ndarray, tol=1e-6) -> np.ndarray:
    x_ = x.copy().astype(float)
    r = b - A @ x_
    v = r.copy()
    for _ in range(v.shape[0]):
        print(x_)
        if np.linalg.norm(v) < tol:
            return x_
        s = 1 / np.dot(r.flatten(), r.flatten())
        t = np.dot(r.flatten(), r.flatten()) / (np.dot(v.flatten(), A @ v))
        x_ += (t * v).reshape(x_.shape)
        r -= t * A @ v
        if np.linalg.norm(r) < tol:
            return x_
        s *= np.dot(r.flatten(), r.flatten())
        v = r + s * v
    return x_

def PreconditionConjugateGradient(A:np.ndarray, C:np.ndarray, x:np.ndarray, b:np.ndarray, tol=1e-6) -> np.ndarray:
    x_ = x.copy().astype(float)
    r = b - A @ x_
    w = C @ r
    v = C.T @ w
    for _ in range(v.shape[0]):
        if np.linalg.norm(v) < tol:
            return x_
        s = 1 / np.dot(w.flatten(), w.flatten())
        t = np.dot(w.flatten(), w.flatten()) / (np.dot(v.flatten(), A @ v))
        x_ += (t * v).reshape(x_.shape)
        r -= t * A @ v
        if np.linalg.norm(r) < tol:
            return x_
        w = C @ r
        s *= np.dot(w.flatten(), w.flatten())
        v = C.T @ w + s * v
    return x_

# A = np.array([
#     [4, 3, 0],
#     [3, 4, -1],
#     [0, -1, 4],
# ])
# b = np.array([
#     [24], [30], [-24]
# ])
# v = np.array([
#     [1, -3 / 4, -3 / 7],
#     [0, 1, 4 / 7],
#     [0, 0, 1]
# ])
# x = np.array([
#     [0], [0], [0]
# ])
# ConjugateGradient(A, x, b, v), ConjugateGradient_(A, x, b), PreconditionConjugateGradient(A, np.eye(3), x, b)

In [92]:
A = np.array([
    [0.2, 0.1, 1, 1, 0],
    [0.1, 4, -1, 1, -1],
    [1, -1, 60, 0, -2],
    [1, 1, 0, 8, 4],
    [0, -1, -2, 4, 700]
])

b = np.array([
    [1],
    [2],
    [3],
    [4],
    [5]
])

x = np.array([
    [0],
    [0],
    [0],
    [0],
    [0]
], dtype=np.float32)

print("Jacobi Method")
print(Jacobi(A, x, b, 100))

print("Gauss-Seidel Method")
print(GaussSeidel(A, x, b, 100))

print("SOR")
print(SOR(A, x, b, 100, 1.25, 0.01))

print("Conjugate Gradient")
print(ConjugateGradient_(A, x, b, 0.01))

print("Preconditioned Conjugate Gradient")
print(PreconditionConjugateGradient(A, 1 / np.sqrt(np.diag(A)) * np.eye(A.shape[0]), x, b, 0.01))

Jacobi Method
[[ 7.859706  ]
 [ 0.42292595]
 [-0.07359236]
 [-0.54064405]
 [ 0.01062616]]
Gauss-Seidel Method
[[ 7.8597126 ]
 [ 0.42292643]
 [-0.07359223]
 [-0.540643  ]
 [ 0.01062616]]
SOR
[[ 7.859713  ]
 [ 0.4229264 ]
 [-0.07359224]
 [-0.54064304]
 [ 0.01062616]]
Conjugate Gradient
0
[[0.00300832]
 [0.00601665]
 [0.00902497]
 [0.0120333 ]
 [0.01504162]]
1
[[0.04647092]
 [0.09363862]
 [0.12985331]
 [0.18422967]
 [0.00632678]]
2
[[0.127921  ]
 [0.26835779]
 [0.05196309]
 [0.493859  ]
 [0.00484866]]
3
[[0.3059927 ]
 [0.49147673]
 [0.05351802]
 [0.38951203]
 [0.00577334]]
4
[[ 7.85971308]
 [ 0.42292641]
 [-0.07359224]
 [-0.54064302]
 [ 0.01062616]]
[[ 7.85971308]
 [ 0.42292641]
 [-0.07359224]
 [-0.54064302]
 [ 0.01062616]]
Preconditioned Conjugate Gradient
0
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
1
[[2.79266985]
 [0.27926699]
 [0.0279267 ]
 [0.27926699]
 [0.00398953]]
2
[[ 5.43426173]
 [ 0.67245844]
 [-0.03550208]
 [-0.24152014]
 [ 0.01235105]]
3
[[ 7.77113748]
 [ 0.43706384]
 [-0.05561341]
 [-0

# Appendix
## Theorem 1
